# Evaluate DFR models on all ImageNet variations

In [73]:
import sys
import numpy as np
import tqdm
import pickle

from matplotlib import pyplot as plt
%matplotlib inline

In [74]:
def load_embeddings(path, mean=None, std=None):
    arr = np.load(path)
    x, y = arr["embeddings"], arr["labels"]
    if mean is not None:
        x = (x - mean) / std
    return x, y

Change data paths here.

In [75]:
imagenet_c_corruptions = ["brightness", "defocus_blur", "fog", "gaussian_blur", "glass_blur",
                          "jpeg_compression", "pixelate", "shot_noise", "spatter", "zoom_blur",
                          "contrast", "elastic_transform", "frost", "gaussian_noise",
                          "impulse_noise", "motion_blur", "saturate", "snow", "speckle_noise"]
intensities = [1, 2, 3, 4, 5]

eval_path_dict = {
    "imagenet_r": "/datasets/imagenet-r/imagenet-r_resnet50_val_embeddings.npz",
    "imagenet_a": "/datasets/imagenet-a/imagenet-a_resnet50_val_embeddings.npz",
    "imagenet": "/datasets/imagenet_symlink/resnet50_val_embeddings.npz",
    "imagenet_stylized": "/datasets/imagenet-stylized/imagenet_resnet50_val_embeddings.npz",
}
eval_path_dict.update({
    f"imagenet_c_{corruption}_{intensity}": 
        f"/datasets/imagenet-c/{corruption}/{intensity}/imagenet-c_resnet50_val_embeddings.npz"
    for corruption in imagenet_c_corruptions for intensity in intensities
})

## DFR-IN-SIN

In [78]:
arr = np.load("dfr_insin_weights_bs10k.npz")
w = arr["w"]
b = arr["b"]
preprocess_mean = arr["preprocess_mean"]
preprocess_std = arr["preprocess_std"]

In [79]:
eval_datasets = {k: load_embeddings(p, mean=preprocess_mean, std=preprocess_std)
                 for k, p in eval_path_dict.items()}

In [80]:
results = {}
for key, (x, y) in tqdm.tqdm(eval_datasets.items()):
    preds = x @ w.T + b[None, :]
    preds_cls = np.argsort(preds, -1)
    preds1 = preds_cls[:, -1:]
    preds5 = preds_cls[:, -5:]
    acc1 = (preds1 == y[:, None]).sum() / len(y)
    acc5 = (preds5 == y[:, None]).sum() / len(y)
    results[key] = (acc1, acc5)
results

100%|██████████| 99/99 [04:38<00:00,  2.81s/it]


{'imagenet_r': (0.27166666666666667, 0.42436666666666667),
 'imagenet_a': (0.0024, 0.039466666666666664),
 'imagenet': (0.74524, 0.9178),
 'imagenet_stylized': (0.21418, 0.39042),
 'imagenet_c_brightness_1': (0.69676, 0.89084),
 'imagenet_c_brightness_2': (0.68142, 0.87932),
 'imagenet_c_brightness_3': (0.65608, 0.86096),
 'imagenet_c_brightness_4': (0.6165, 0.83204),
 'imagenet_c_brightness_5': (0.56082, 0.7897),
 'imagenet_c_defocus_blur_1': (0.5479, 0.78022),
 'imagenet_c_defocus_blur_2': (0.4714, 0.71306),
 'imagenet_c_defocus_blur_3': (0.32916, 0.55972),
 'imagenet_c_defocus_blur_4': (0.2244, 0.42184),
 'imagenet_c_defocus_blur_5': (0.14804, 0.30394),
 'imagenet_c_fog_1': (0.5928, 0.8149),
 'imagenet_c_fog_2': (0.54144, 0.77306),
 'imagenet_c_fog_3': (0.46918, 0.70442),
 'imagenet_c_fog_4': (0.42214, 0.65012),
 'imagenet_c_fog_5': (0.2852, 0.48892),
 'imagenet_c_gaussian_blur_1': (0.63952, 0.8516),
 'imagenet_c_gaussian_blur_2': (0.50584, 0.74432),
 'imagenet_c_gaussian_blur_3': (

## DFR-SIN

In [86]:
arr = np.load("dfr_sin_weights_bs10k.npz")
w = arr["w"]
b = arr["b"]
preprocess_mean = arr["preprocess_mean"]
preprocess_std = arr["preprocess_std"]

In [87]:
eval_datasets = {k: load_embeddings(p, mean=preprocess_mean, std=preprocess_std)
                 for k, p in eval_path_dict.items()}

In [88]:
results = {}
for key, (x, y) in tqdm.tqdm(eval_datasets.items()):
    preds = x @ w.T + b[None, :]
    preds_cls = np.argsort(preds, -1)
    preds1 = preds_cls[:, -1:]
    preds5 = preds_cls[:, -5:]
    acc1 = (preds1 == y[:, None]).sum() / len(y)
    acc5 = (preds5 == y[:, None]).sum() / len(y)
    results[key] = (acc1, acc5)
results

100%|██████████| 99/99 [04:26<00:00,  2.69s/it]


{'imagenet_r': (0.24566666666666667, 0.3939666666666667),
 'imagenet_a': (0.004, 0.04),
 'imagenet': (0.65076, 0.85662),
 'imagenet_stylized': (0.21952, 0.39462),
 'imagenet_c_brightness_1': (0.62642, 0.83606),
 'imagenet_c_brightness_2': (0.60964, 0.82278),
 'imagenet_c_brightness_3': (0.58636, 0.80426),
 'imagenet_c_brightness_4': (0.5512, 0.77514),
 'imagenet_c_brightness_5': (0.50192, 0.73456),
 'imagenet_c_defocus_blur_1': (0.49322, 0.72342),
 'imagenet_c_defocus_blur_2': (0.42214, 0.6562),
 'imagenet_c_defocus_blur_3': (0.28902, 0.50306),
 'imagenet_c_defocus_blur_4': (0.19266, 0.3718),
 'imagenet_c_defocus_blur_5': (0.125, 0.2662),
 'imagenet_c_fog_1': (0.54142, 0.76494),
 'imagenet_c_fog_2': (0.4977, 0.72516),
 'imagenet_c_fog_3': (0.43576, 0.66294),
 'imagenet_c_fog_4': (0.39164, 0.61172),
 'imagenet_c_fog_5': (0.26734, 0.46046),
 'imagenet_c_gaussian_blur_1': (0.57676, 0.79614),
 'imagenet_c_gaussian_blur_2': (0.4556, 0.688),
 'imagenet_c_gaussian_blur_3': (0.3344, 0.5578),
 

## DFR-IN

In [90]:
arr = np.load("dfr_in_weights_bs10k.npz")
w = arr["w"]
b = arr["b"]
preprocess_mean = arr["preprocess_mean"]
preprocess_std = arr["preprocess_std"]

In [91]:
eval_datasets = {k: load_embeddings(p, mean=preprocess_mean, std=preprocess_std)
                 for k, p in eval_path_dict.items()}

In [92]:
results = {}
for key, (x, y) in tqdm.tqdm(eval_datasets.items()):
    preds = x @ w.T + b[None, :]
    preds_cls = np.argsort(preds, -1)
    preds1 = preds_cls[:, -1:]
    preds5 = preds_cls[:, -5:]
    acc1 = (preds1 == y[:, None]).sum() / len(y)
    acc5 = (preds5 == y[:, None]).sum() / len(y)
    results[key] = (acc1, acc5)
results

100%|██████████| 99/99 [07:11<00:00,  4.36s/it]


{'imagenet_r': (0.2287, 0.3642),
 'imagenet_a': (0.0004, 0.0324),
 'imagenet': (0.75224, 0.92474),
 'imagenet_stylized': (0.06264, 0.1439),
 'imagenet_c_brightness_1': (0.69752, 0.89402),
 'imagenet_c_brightness_2': (0.6761, 0.88078),
 'imagenet_c_brightness_3': (0.64328, 0.85982),
 'imagenet_c_brightness_4': (0.59238, 0.8237),
 'imagenet_c_brightness_5': (0.52192, 0.76648),
 'imagenet_c_defocus_blur_1': (0.53122, 0.77282),
 'imagenet_c_defocus_blur_2': (0.45026, 0.69792),
 'imagenet_c_defocus_blur_3': (0.30214, 0.53054),
 'imagenet_c_defocus_blur_4': (0.19806, 0.38572),
 'imagenet_c_defocus_blur_5': (0.1242, 0.26832),
 'imagenet_c_fog_1': (0.55736, 0.7949),
 'imagenet_c_fog_2': (0.49118, 0.73648),
 'imagenet_c_fog_3': (0.40022, 0.64656),
 'imagenet_c_fog_4': (0.33862, 0.57352),
 'imagenet_c_fog_5': (0.19868, 0.38268),
 'imagenet_c_gaussian_blur_1': (0.6289, 0.85092),
 'imagenet_c_gaussian_blur_2': (0.48582, 0.73352),
 'imagenet_c_gaussian_blur_3': (0.3359, 0.57272),
 'imagenet_c_gauss